# **ENRON Dataset Preprocesing**

This notebook:
1. installs an email dataset from Kaggle
2. parses the javascript representation of the email to then extract the body of the message
3.  \(attempts to) cleans the data of filler words and typos
4. saves the bag of words matrix representation to a parquet file



## 1. Installing the data

Installs the dataset from Kaggle, other dataset should be able to be loaded in by replacing the 2nd code block in this section.

We also only look at the first 5000 emails. This value can be adjusted in the 3rd code block

In [1]:
# all necessary import statements
import email
import pandas as pd

In [2]:
# THIS CODE BLOCK IS FROM KAGGLE
# THIS IS SPECIFIC FOR THE ENRON DATASET, THIS CAN BE REPLACED TO USE OTHER DATASETS

# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "emails.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "wcukierski/enron-email-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipython-input-594570759.py:13: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 358M/358M [00:06<00:00, 60.4MB/s]

Extracting zip of emails.csv...


First 5 records:                        file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
df = df.iloc[::100]
print(len(df))

5175


##2. Parsing Email

The following function separates out the sender, recipient, date sent, subject, and body of the emails and saves the results to the "parsed_emails" pandas dataframe.

In [ ]:
def parse_email(message: str):
  """
  TODO: docstring
  """
  try:
      msg = email.message_from_string(message)
      sender = msg['from']
      recipient = msg['to']
      date = msg['date']
      subject = msg['subject']
      body = ''

      if msg.is_multipart():
          body = ""
          for part in msg.walk():
              if part.get_content_type() == "text/plain":
                  body += part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8')
      else:
          body = msg.get_payload(decode=True).decode(msg.get_content_charset() or 'utf-8')

      return pd.Series([sender, recipient, date, subject, body])

  except Exception as e:
      print(f"Error parsing email: {e}")
      return pd.Series([None, None, None, None, None])


parsed_emails = df['message'].apply(parse_email)
display(parsed_emails.iloc[:20])
print(len(parsed_emails))

,0,1,2,3,4
0,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Here is our forecast\n\n
1,phillip.allen@enron.com,john.lavorato@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,leah.arsdall@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,"Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,greg.piper@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,Let's shoot for Tuesday at 11:45.
5,phillip.allen@enron.com,greg.piper@enron.com,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,"Greg,\n\n How about either next Tuesday or Thu..."
6,phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com","Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",,Please cc the following distribution list with...
7,phillip.allen@enron.com,joyce.teixeira@enron.com,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",Re: PRC review - phone calls,any morning between 10 and 11:30
8,phillip.allen@enron.com,mark.scott@enron.com,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...
9,phillip.allen@enron.com,zimam@enron.com,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...


517401


In [ ]:
parsed_emails.columns = ['sender', 'recipient', 'date', 'subject', 'body']
display(parsed_emails.head())

,sender,recipient,date,subject,body
0,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Here is our forecast\n\n
100,phillip.allen@enron.com,cooper.richey@enron.com,"Wed, 16 Aug 2000 05:35:00 -0700 (PDT)",Re:,I tried the new address but I don't have acces...
200,phillip.allen@enron.com,mike.grigsby@enron.com,"Thu, 6 Apr 2000 05:18:00 -0700 (PDT)",Alliance netback worksheet,---------------------- Forwarded by Phillip K ...
300,phillip.allen@enron.com,andrea.richards@enron.com,"Mon, 23 Apr 2001 13:40:00 -0700 (PDT)",Re: Analyst Bryan Hull,"Andrea,\n\nAfter reviewing Bryan Hull's resume..."
400,phillip.allen@enron.com,rlehmann@yahoo.com,"Wed, 7 Mar 2001 04:09:00 -0800 (PST)",,"Reagan,\n\nHere is the cost estimate and profo..."


In [ ]:
parsed_emails.to_parquet("parsed_email.parquet", engine='pyarrow', compression='snappy')

##3. Data Cleaning

The following code removes filler words, determined using the nltk set of "stopwords".

It also removes any punctuation, numbers, unnecessary sender information and attempts to remove frequent typos.

This section may need to be tweaked and hardcode in some more rules.

In [ ]:
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_email_body(text: str):
  """
  TODO: docstring
  """
  text = text.lower()
  text = re.sub(r'\s+', ' ', text)
  text = text.translate(str.maketrans('', '', punctuation))

  # 4. Remove stopwords (common filler words)
  stop_words = set(stopwords.words('english'))
  words = [word for word in text.split() if word not in stop_words]

  return words

def remove_email_artifacts(tokens: list[str]):
  """
  TODO: docstring
  """
  # Common email / header substrings to remove if they appear anywhere inside a token
  junk_patterns = [
      r'enron', r'xgate', r'ect', r'com', r'mail', r'message', r'id',
      r'thyme', r'pst', r'sent', r'nonprivileged', r'folder', r'origin'
  ]
  clean = []
  for t in tokens:
      # Remove tokens containing junk substrings (case-insensitive)
      if any(re.search(pat, t, re.IGNORECASE) for pat in junk_patterns):
          continue
      if any(ch.isdigit() for ch in t):
          continue
      if len(t) < 3 or len(t) > 15:
          continue
      clean.append(t)
  return clean

In [ ]:
parsed_emails['filtered_body'] = parsed_emails['body'].apply(clean_email_body)
parsed_emails['cleaned_body'] = parsed_emails['filtered_body'].apply(remove_email_artifacts)
display(parsed_emails.head())

,sender,recipient,date,subject,body,filtered_body,cleaned_body
0,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Here is our forecast\n\n,[forecast],[forecast]
100,phillip.allen@enron.com,cooper.richey@enron.com,"Wed, 16 Aug 2000 05:35:00 -0700 (PDT)",Re:,I tried the new address but I don't have acces...,"[tried, new, address, dont, access, also, need...","[tried, new, address, dont, access, also, need..."
200,phillip.allen@enron.com,mike.grigsby@enron.com,"Thu, 6 Apr 2000 05:18:00 -0700 (PDT)",Alliance netback worksheet,---------------------- Forwarded by Phillip K ...,"[forwarded, phillip, k, allenhouect, 04062000,...","[forwarded, phillip, julie, gomez, hunter, phi..."
300,phillip.allen@enron.com,andrea.richards@enron.com,"Mon, 23 Apr 2001 13:40:00 -0700 (PDT)",Re: Analyst Bryan Hull,"Andrea,\n\nAfter reviewing Bryan Hull's resume...","[andrea, reviewing, bryan, hulls, resume, thin...","[andrea, reviewing, bryan, hulls, resume, thin..."
400,phillip.allen@enron.com,rlehmann@yahoo.com,"Wed, 7 Mar 2001 04:09:00 -0800 (PST)",,"Reagan,\n\nHere is the cost estimate and profo...","[reagan, cost, estimate, proforma, prepared, g...","[reagan, cost, estimate, proforma, prepared, g..."


##3. Bag of words tokenization

This section saves the cleaned email body words into a sparse matrix.
Due to time complexity of saving a large matrix, we save this matrix into a parquet file.

Other important information like "sender", "recipient", "subject", are currently not saved to the parquet file

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(parsed_emails['cleaned_body'])
vocab = vectorizer.get_feature_names_out()

bow_df = pd.DataFrame(X.toarray(), columns=vocab)

display(bow_df.head())

# Save as Parquet (fast and memory-efficient compared to CSV)
bow_df.to_parquet("word_matrix.parquet", engine='pyarrow', compression='snappy')

,emerging,any,arising,born,bulletin,but,center,decisions,dedicated,economic,...,zur,zuraski,zurich,zusaetzlich,zuzana,zuzka,zwerdling,zwischen,zychers,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
